In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from datetime import datetime
from vnstock import *
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from copy import deepcopy as dc
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error



In [3]:
# config
stock_target='VGI'
lookback = 5
origin_start_date= datetime(2020, 1, 2)
start_date = datetime(2023, 8, 20)
end_date = datetime(2023, 12, 25)

#### Util


In [4]:
def move_day(today): 
    if today.weekday() == 5:  # Saturday
     today += timedelta(days=2)
    elif today.weekday() == 6:  # Sunday
     today += timedelta(days=1)
    return today

def generate_date_range(start_date, end_date):
    # mapping to stock is that day exist 
    df_map = stock_historical_data(symbol=stock_target, start_date=start_date.strftime('%Y-%m-%d'),end_date=end_date.strftime('%Y-%m-%d'), resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')
        
    return pd.to_datetime(df_map['time'])


def restore_df(X_scaler,y_scaler,scaler):
   
   num_features = 0
   # (number samples, number time steps, number features)
   if X_scaler.shape[1] == 1:
      num_features = X_scaler.shape[2]
   elif  X_scaler.shape[2] == 1:
      num_features = X_scaler.shape[1]  

   dummies = np.zeros((X_scaler.shape[0],num_features + 1))
   if  X_scaler.shape[1] == 1:
        # don't use timestep instead use features
       dummies[:,1:] = X_scaler.reshape((X_scaler.shape[0],X_scaler.shape[2]))
   elif X_scaler.shape[1] != 1:
       # use time steps
       dummies[:,1:] = X_scaler.reshape((X_scaler.shape[0],X_scaler.shape[1]))
   dummies[:, 0] = y_scaler.flatten()
   df = scaler.inverse_transform(dummies)
   return df

def get_predict_and_df(model, X, scaler):
    if isinstance(X, np.ndarray):
        X = torch.tensor(X).float() 
    
    with torch.no_grad():
       y_predicted = model(X.to(device)).to('cpu').numpy()
      
    y_predict = y_predicted.flatten()
    df = restore_df(X, y_predict, scaler)
    return df
   
def get_predict(model, X, scaler):
    with torch.no_grad():
      df = get_predict_and_df(model, X, scaler)
      return df[:,:1].reshape(df.shape[0])
    
def get_predict_next_day(model,scaler,shifted_df_as_np_scaler, lag=60):
    X_predict_next_day = create_X_predict_next_day(shifted_df_as_np_scaler,lag) 
    X = create_X_LSTM(X_predict_next_day)
    y_predict = get_predict(model, X, scaler)[-1:]
    return y_predict
    
def create_X_LSTM(X):
    X_flip = dc(np.flip(X, axis = 1))
    X_flip_3D = X_flip.reshape((-1, lookback, 1))
    return X_flip_3D
    
def create_X_predict_next_day(shifted_df_as_np_scaler ,lag=60):
    lag_X = shifted_df_as_np_scaler[-lag:,1:]
    latest_X = shifted_df_as_np_scaler[-1:,:lookback]
    return create_X_LSTM(np.append(lag_X,latest_X,axis=0))

    
def create_scaler_df(df):
     df_np = df.to_numpy()
     scaler = MinMaxScaler(feature_range = (0, 1))
     df_np_scaler = scaler.fit_transform(df_np)
     return df_np_scaler, scaler

def multi_scaler_df(df_list):
    df_np_scaler_list = []
    scaler_list = []
    for df in df_list:    
      df_np_scaler, scaler = create_scaler_df(df)
      df_np_scaler_list.append(df_np_scaler)
      scaler_list.append(scaler)
    return df_np_scaler_list, scaler_list
            

In [5]:
day_test_range = generate_date_range(start_date,end_date)

In [6]:
len(day_test_range)

88

In [7]:
def get_list_next_day_actual_test_range():
    begin_day = (day_test_range.iloc[0] + timedelta(days=1)).strftime('%Y-%m-%d')
    stop_day = (day_test_range.iloc[-1] + timedelta(days=1)).strftime('%Y-%m-%d')
    df_raw = stock_historical_data(symbol=stock_target, start_date=begin_day,end_date=stop_day, resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')
    return df_raw

In [8]:
df_fw_list = []
print("Get stock data from origin data 2020-01-02 to latest test day "+ end_date.strftime('%Y-%m-%d'))
df_raw = stock_historical_data(symbol=stock_target, start_date=origin_start_date.strftime('%Y-%m-%d') ,end_date=end_date.strftime('%Y-%m-%d'), resolution="1D", type="stock", beautify=True, decor=False, source='DNSE')
df_fw_gen = pd.DataFrame()
df_fw_gen[['ds','y']] = df_raw[['time','close']]
df_fw_gen['ds'] = pd.to_datetime(df_fw_gen['ds'] )

for upper_bound_date in  day_test_range:
    df_fw = df_fw_gen[ df_fw_gen['ds'] <= upper_bound_date]
    print(f"Create train set from {origin_start_date.strftime('%Y-%m-%d')} to {upper_bound_date.strftime('%Y-%m-%d')}")
    df_fw_list.append(df_fw)


Get stock data from origin data 2020-01-02 to latest test day 2023-12-25
Create train set from 2020-01-02 to 2023-08-21
Create train set from 2020-01-02 to 2023-08-22
Create train set from 2020-01-02 to 2023-08-23
Create train set from 2020-01-02 to 2023-08-24
Create train set from 2020-01-02 to 2023-08-25
Create train set from 2020-01-02 to 2023-08-28
Create train set from 2020-01-02 to 2023-08-29
Create train set from 2020-01-02 to 2023-08-30
Create train set from 2020-01-02 to 2023-08-31
Create train set from 2020-01-02 to 2023-09-05
Create train set from 2020-01-02 to 2023-09-06
Create train set from 2020-01-02 to 2023-09-07
Create train set from 2020-01-02 to 2023-09-08
Create train set from 2020-01-02 to 2023-09-11
Create train set from 2020-01-02 to 2023-09-12
Create train set from 2020-01-02 to 2023-09-13
Create train set from 2020-01-02 to 2023-09-14
Create train set from 2020-01-02 to 2023-09-15
Create train set from 2020-01-02 to 2023-09-18
Create train set from 2020-01-02 t

In [9]:
list_model = []
shifted_df_list = []

In [10]:
def prepare_dataframe_for_lstm(df, n_steps):
  df = dc(df)
  df['ds'] = pd.to_datetime(df['ds'])

  df.set_index('ds', inplace = True)

  for i in range(1, n_steps + 1):
    df[f'y(t-{i})'] = df['y'].shift(i)
  df.dropna(inplace = True)

  return df


for df_fw in df_fw_list:
   shifted_df  = prepare_dataframe_for_lstm(df_fw,lookback)
   shifted_df_list.append(shifted_df)


In [11]:
shifted_df_as_np_scaler_list, scaler_list = multi_scaler_df(shifted_df_list)

In [12]:
X_list = []
y_list = []

shifted_df_as_np_scaler_list, scaler_list = multi_scaler_df(shifted_df_list)

for shifted_df_as_np_scaler in shifted_df_as_np_scaler_list:
    X = shifted_df_as_np_scaler[:, 1:]
    y = shifted_df_as_np_scaler[:, 0:1]
    X_list.append(X)
    y_list.append(y)


X_list[0].shape, y_list[0].shape

((904, 5), (904, 1))

In [13]:
X_list = [dc(np.flip(X, axis = 1)) for X in X_list]

X_list[0].shape

(904, 5)

In [14]:
X_train_list = X_list
y_train_list = y_list

X_train_list[0].shape, y_train_list[0].shape

((904, 5), (904, 1))

In [15]:
X_train_list = [X_train.reshape((-1, lookback, 1)) for X_train in X_train_list]
y_train_list =  [y_train.reshape((-1, 1)) for y_train in y_train_list]

X_train_list[0].shape, y_train_list[0].shape

((904, 5, 1), (904, 1))

In [16]:
X_train_list = [torch.tensor(X_train).float() for X_train in X_train_list]
y_train_list = [torch.tensor(y_train).float() for y_train in y_train_list]

X_train_list[0].shape, y_train_list[0].shape

(torch.Size([904, 5, 1]), torch.Size([904, 1]))

In [17]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, i):
    return self.X[i], self.y[i]


train_dataset_list = [TimeSeriesDataset(X_train, y_train) for X_train, y_train in zip(X_train_list,y_train_list)]

In [18]:
train_dataset_list[0]

In [19]:
from torch.utils.data import DataLoader

batch_size = 12

train_loader_list = [DataLoader(train_dataset, batch_size = batch_size, shuffle = True) for train_dataset in train_dataset_list ]

In [20]:
class LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_stacked_layers):
    super().__init__()
    self.hidden_size = hidden_size
    self.num_stacked_layers = num_stacked_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers,
                        batch_first = True)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self, x):
    batch_size = x.size(0)
    h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
    c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
    out, _ = self.lstm(x, (h0, c0))
    out = self.fc(out[:, -1, :])
    return out



for i in range(len(train_loader_list)):
  m = LSTM(1, 4, 1) #1,4,1 19:30
  m.to(device)
  list_model.append(m)


In [21]:
learning_rate = 0.001
num_epochs = 20
loss_function = nn.MSELoss()

def train_one_epoch(model,train_loader,epoch):
  optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
  model.train(True)
  print(f'Epoch: {epoch + 1}')
  running_loss = 0.0

  for batch_index, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)
    loss = loss_function(output, y_batch)
    running_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch_index % 20 == 10: # print every 10 batches
        avg_loss_across_batches = running_loss / 1
        print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                              avg_loss_across_batches))
        running_loss = 0.0
  print()

In [22]:
for model, train_loader in zip(list_model,train_loader_list):
  for epoch in range(num_epochs):
    train_one_epoch(model=model,train_loader=train_loader,epoch=epoch)

Epoch: 1
Batch 11, Loss: 10.914
Batch 31, Loss: 18.934
Batch 51, Loss: 17.507
Batch 71, Loss: 14.841

Epoch: 2
Batch 11, Loss: 7.932
Batch 31, Loss: 11.956
Batch 51, Loss: 9.882
Batch 71, Loss: 9.138

Epoch: 3
Batch 11, Loss: 4.554
Batch 31, Loss: 6.776
Batch 51, Loss: 6.305
Batch 71, Loss: 5.577

Epoch: 4
Batch 11, Loss: 2.243
Batch 31, Loss: 3.431
Batch 51, Loss: 2.540
Batch 71, Loss: 1.589

Epoch: 5
Batch 11, Loss: 0.709
Batch 31, Loss: 0.859
Batch 51, Loss: 0.633
Batch 71, Loss: 0.599

Epoch: 6
Batch 11, Loss: 0.394
Batch 31, Loss: 0.544
Batch 51, Loss: 0.485
Batch 71, Loss: 0.531

Epoch: 7
Batch 11, Loss: 0.330
Batch 31, Loss: 0.403
Batch 51, Loss: 0.377
Batch 71, Loss: 0.325

Epoch: 8
Batch 11, Loss: 0.192
Batch 31, Loss: 0.346
Batch 51, Loss: 0.206
Batch 71, Loss: 0.211

Epoch: 9
Batch 11, Loss: 0.117
Batch 31, Loss: 0.170
Batch 51, Loss: 0.119
Batch 71, Loss: 0.108

Epoch: 10
Batch 11, Loss: 0.049
Batch 31, Loss: 0.051
Batch 51, Loss: 0.048
Batch 71, Loss: 0.039

Epoch: 11
Batc

In [23]:
len(df_fw_list[0])

909

In [24]:
len(X_train_list[0])

904

In [25]:
df = df_fw_list[0]
X_train = X_train_list[0]
scaler = scaler_list[0]

fig = go.Figure()


actual_y_scat = go.Scatter(x=df['ds'][lookback:], y=df['y'][lookback:], mode='lines', name="y", line=dict(color='red'))
predicted_y = get_predict(model=model,X=X_train, scaler=scaler)
predict_y_scat = go.Scatter(x=df['ds'][lookback:], y=predicted_y, mode='lines', name='yhat', line=dict(color='blue'))
fig.add_trace(actual_y_scat)
fig.add_trace(predict_y_scat)

# Adjust layout for better spacing
fig.update_layout(title_text=f"Training forward form {origin_start_date.strftime('%Y-%m-%d')} up upper bound date")
fig.update_xaxes(title_text='Date')  # Set X-axis label for the bottom subplot
fig.update_yaxes(title_text='Price')  # Set Y-axis label for the left subplot
# Show the figure
fig.show()

#### Forward Evaluate Model

In [26]:
def get_list_next_day_predict_test_range():
    next_days = []
    for model, scaler, shifted_df_as_np_scaler in zip(list_model, scaler_list, shifted_df_as_np_scaler_list):
      next_day_predict = get_predict_next_day(model, scaler, shifted_df_as_np_scaler,1)
      next_days.append(next_day_predict)
    
    return np.array(next_days).reshape(len(next_days))


def create_evalute_table():
    next_days = get_list_next_day_predict_test_range()
    df = get_list_next_day_actual_test_range()
    df['predict'] = pd.Series(next_days)
    df[['ds','y','yhat']] = df[['time','close','predict']]
    df = df[['ds','y','yhat']]
    df['ds'] = pd.to_datetime(df['ds'])
    return df
   

In [27]:
df_eval = create_evalute_table()
df_eval['y_change'] = df_eval['y'].diff()
df_eval['yhat_change'] = df_eval['yhat'].diff()
df_eval['y*yhat_chage'] = df_eval['y_change'] * df_eval['yhat_change']

In [28]:
df_eval

,ds,y,yhat,y_change,yhat_change,y*yhat_chage
0,2023-08-22,23700,25332.029760,NaN,NaN,NaN
1,2023-08-23,23500,25208.113492,-200.0,-123.916268,24783.253670
2,2023-08-24,24000,24476.657212,500.0,-731.456280,-365728.139877
3,2023-08-25,24200,23730.296791,200.0,-746.360421,-149272.084236
4,2023-08-28,24700,23472.865522,500.0,-257.431269,-128715.634346
...,...,...,...,...,...,...
83,2023-12-20,25200,25333.217978,200.0,-579.109490,-115821.897984
84,2023-12-21,25100,25494.475961,-100.0,161.257982,-16125.798225
85,2023-12-22,25500,25048.005283,400.0,-446.470678,-178588.271141
86,2023-12-25,25800,24781.095088,300.0,-266.910195,-80073.058605


In [30]:
from sklearn.metrics import r2_score

In [31]:
# Calculate RMSE
rmse = np.sqrt(mean_squared_error(df_eval['y'], df_eval['yhat']))
print(f"Root Mean Squared Error (RMSE): {rmse}")

# Calculate correlation coefficient
correlation = df_eval['y'].corr(df_eval['yhat'])
print(f"Correlation Coefficient: {correlation}")

truth = []
for y_hat_change, y_change in zip(df_eval['yhat'].diff().dropna(),df_eval['y'].diff().dropna()):
    truth.append(1 if y_hat_change * y_change > 0 else 0)


print(f"Direction correction ratio: {np.sum(truth)/len(truth) * 100}")

r2 = r2_score(df_eval['y'], df_eval['yhat'])
print(f"R-squared (R²): {r2}")

Root Mean Squared Error (RMSE): 1268.2172334667778
Correlation Coefficient: 0.6347027061441937
Direction correction ratio: 40.229885057471265
R-squared (R²): 0.3210056203240619


In [33]:
df_mtr = pd.read_csv('./output/metrics-VGI.csv')
# remove old record
df_mtr = df_mtr[~(df_mtr['model'] == 'LSTM(1,4,1)')]

df_mtr = pd.concat([df_mtr,
                    pd.DataFrame({
                     'model': ['LSTM(1,4,1)'],
                     'rmse': [rmse],
                     'correlation': [correlation],
                     'r-squared': [r2] })], axis=0)
df_mtr.to_csv('./output/metrics-VGI.csv',index=False)

df_mtr

,model,rmse,r-squared,correlation
0,"ARIMA(9, 1, 9)",935.032698,0.824663,0.912103
1,Prophet,1493.068399,-0.001789,0.304553
0,"LSTM(1,4,1)",1268.217233,0.321006,0.634703


In [ ]:
# Plotting with Plotly
fig = go.Figure()

# Plot yhat as a line
fig.add_trace(go.Scatter(x=df_eval['ds'], y=df_eval['yhat'], mode='lines+markers', name='yhat', line=dict(color='blue')))

# Plot y as points
fig.add_trace(go.Scatter(x=df_eval['ds'], y=df_eval['y'], mode='lines+markers', name='y', marker=dict(color='red', size=8)))

# Set layout
fig.update_layout(title=f'{stock_target} moving forward train and predict the next day',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  showlegend=True)

# Show the plot
fig.show()

In [ ]:
marker_colors = ['red' if value < 0 else 'green' for value in df_eval['y*yhat_chage']]

trace = go.Scatter(x=df['ds'], y=df_eval['y*yhat_chage'], mode='lines+markers', name='Result Array', marker=dict(color=marker_colors))

# Create layout
layout = go.Layout(title='Line with Markers for Result Array', xaxis=dict(title='Index'), yaxis=dict(title='Value'))

# Create figure and add trace
fig = go.Figure(data=[trace], layout=layout)

# Show the plot
fig.show()

In [ ]:
import plotly.express as px

scaler_corr = MinMaxScaler(feature_range=(-1, 1)) 

# Here we use a column with categorical data
fig = px.histogram(scaler_corr.fit_transform(df_eval['y*yhat_chage'].to_numpy().reshape(-1, 1)), nbins=60)
fig.show()

In [ ]:
df_eval['y_rolling_7_std'] = df_eval['y_change'].rolling(window=7).std()
df_eval['yhat_rolling_7_std'] = df_eval['yhat_change'].rolling(window=7).std()
df_eval['y*yhat_chage_rolling_7'] =  df_eval['y*yhat_chage'].rolling(window=7).mean()
df_eval['moving_corr'] = df_eval['y*yhat_chage_rolling_7'] / (df_eval['y_rolling_7_std'] * df_eval['yhat_rolling_7_std'])

In [ ]:
scaler_corr = MinMaxScaler(feature_range=(-1, 1)) 

# Here we use a column with categorical data
fig = px.histogram(df_eval['moving_corr'], nbins=60)
fig.show()

In [ ]:
df_eval

,ds,y,yhat,y_change,yhat_change,y*yhat_chage,y_rolling_7_std,yhat_rolling_7_std,y*yhat_chage_rolling_7,moving_corr
0,2023-08-22,23700,25111.304522,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-08-23,23500,25091.869235,-200.0,-19.435287,3887.057304,NaN,NaN,NaN,NaN
2,2023-08-24,24000,24466.856420,500.0,-625.012815,-312506.407499,NaN,NaN,NaN,NaN
3,2023-08-25,24200,23886.197060,200.0,-580.659360,-116131.871939,NaN,NaN,NaN,NaN
4,2023-08-28,24700,23945.922256,500.0,59.725195,29862.597585,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
83,2023-12-20,25200,25430.547297,200.0,-157.158673,-31431.734562,386.066858,380.851118,-18657.739673,-0.126894
84,2023-12-21,25100,25105.083585,-100.0,-325.463712,32546.371222,382.348632,154.373690,15416.609389,0.261190
85,2023-12-22,25500,25035.044849,400.0,-70.038736,-28015.494347,369.684550,163.257534,-16852.102109,-0.279222
86,2023-12-25,25800,24622.087032,300.0,-412.957817,-123887.345195,386.683087,160.887134,-34550.294280,-0.555361
